In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dask.dataframe as dd
import pyspark
import csv
from datetime import date, datetime, timedelta
from itertools import groupby
from operator import itemgetter
import geopandas as gpd
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.mixture import GaussianMixture

In [ ]:
# download dataset

# !wget https://data.cityofnewyork.us/api/views/tqtj-sjs8/rows.csv?accessType=DOWNLOAD
# !wget https://data.cityofnewyork.us/api/views/h9gi-nx95/rows.csv?accessType=DOWNLOAD&bom=true&query=select+*
# !wget https://data.cityofnewyork.us/api/views/tqtj-sjs8/rows.csv?accessType=DOWNLOAD

In [ ]:
# motor_df = pd.read_csv('./data/Motor_Vehicle_Collisions_-_Person.csv')

In [ ]:
# motor_df.head()

In [ ]:
# motor_df['PED_LOCATION'].unique()

In [ ]:
# missingValue = motor_df.isna().sum().to_frame(name='NaN_number')
# missingValue.sort_values('NaN_number',inplace=True,ascending=False)
# missingValue.plot.bar(color='blue',figsize=(15,7))
# plt.title('Missing Value of Motor Vehicle Collisions dataset',fontsize=15)
# plt.legend()
# plt.show()

In [ ]:
crash_df = pd.read_csv('./data/Motor_Vehicle_Collisions_-_Crashes.csv',dtype={'BOROUGH':str,'ZIP CODE':str})

In [ ]:
crash_df.columns.unique().to_list()

In [ ]:
crash_df['CRASH DATE'] = pd.to_datetime(crash_df['CRASH DATE'])
crash_df['CRASH DATE'].sort_values(ascending=False)
crash_df['dow'] = pd.to_datetime(crash_df['CRASH DATE']).dt.dayofweek
crash_df['day'] = pd.to_datetime(crash_df['CRASH DATE']).dt.day
crash_df['year'] = pd.to_datetime(crash_df['CRASH DATE']).dt.year

In [ ]:
# crash_df['Killed'] = crash_df['NUMBER OF PERSONS KILLED']+\
#                         crash_df['NUMBER OF PEDESTRIANS KILLED']+crash_df['NUMBER OF CYCLIST KILLED']
# crash_df['Injured'] = crash_df['NUMBER OF PERSONS INJURED']+\
#                         crash_df['NUMBER OF PEDESTRIANS INJURED']+crash_df['NUMBER OF CYCLIST INJURED']

In [ ]:
crash_df.head()

In [ ]:
crash_df.isna().sum()

In [ ]:
# TODO: Use latitude and longitude to get the borough name
crash = crash_df[crash_df['LATITUDE'].notna()].copy()

In [ ]:
crash['ON STREET NAME'].fillna(crash['CROSS STREET NAME'], inplace=True)
crash['ON STREET NAME'].fillna(crash['OFF STREET NAME'], inplace=True)

In [ ]:
crash.isna().sum()

In [ ]:
crash = crash[(crash['ON STREET NAME'].notna())&(crash['NUMBER OF PERSONS INJURED'].notna())&(crash['NUMBER OF PEDESTRIANS KILLED'].notna())]

In [ ]:
crash.isna().sum()

In [ ]:
geo_df = gpd.read_file('https://data.cityofnewyork.us/api/geospatial/d3c5-ddgc?method=export&format=Shapefile')
geo_df.head()

In [ ]:
gdf_crash = gpd.GeoDataFrame(crash,crs='EPSG:4326',geometry=gpd.points_from_xy(crash.LONGITUDE, crash.LATITUDE))

In [ ]:
crash_temp = gpd.sjoin(gdf_crash,geo_df,how='left',op='intersects')

In [ ]:
crash.shape

In [ ]:
crash_temp.shape

In [ ]:
crash_temp['BOROUGH'].fillna(crash_temp.borough,inplace=True)

In [ ]:
crash_temp.BOROUGH.isna().sum()

In [ ]:
crash_temp = crash_temp[crash_temp.BOROUGH.notna()].copy()

In [ ]:
crash_temp.isna().sum()

In [ ]:
crash_temp['CONTRIBUTING FACTOR VEHICLE 1'].unique()

In [ ]:
crash_final = crash_temp.copy()

In [ ]:
crash_final['ON STREET NAME'] = crash_final['ON STREET NAME'].apply(lambda x: x.strip())
crash_final['BOROUGH'] = crash_final['BOROUGH'].str.upper()

In [ ]:
crash_final['key'] = list(zip(crash_final['BOROUGH'],crash_final['ON STREET NAME']))

In [ ]:
crash_final.head()

In [ ]:
crash_final.columns.to_list()

In [ ]:
save_column = ['CRASH DATE',
 'CRASH TIME',
 'BOROUGH',
 'ZIP CODE',
 'ON STREET NAME',
 'NUMBER OF PERSONS INJURED',
 'NUMBER OF PERSONS KILLED',
 'NUMBER OF PEDESTRIANS INJURED',
 'NUMBER OF PEDESTRIANS KILLED',
 'NUMBER OF CYCLIST INJURED',
 'NUMBER OF CYCLIST KILLED',
 'NUMBER OF MOTORIST INJURED',
 'NUMBER OF MOTORIST KILLED',
 'COLLISION_ID',
 'dow',
 'day',
 'year',
 'geometry',
 'key']

In [ ]:
# crash_final = crash_final[save_column].copy()

In [ ]:
# crash_final = crash_final[['key', 'NUMBER OF PERSONS INJURED',
#  'NUMBER OF PERSONS KILLED',
#  'NUMBER OF PEDESTRIANS INJURED',
#  'NUMBER OF PEDESTRIANS KILLED',
#  'NUMBER OF CYCLIST INJURED',
#  'NUMBER OF CYCLIST KILLED',
#  'NUMBER OF MOTORIST INJURED',
#  'NUMBER OF MOTORIST KILLED']].groupby('key').sum().reset_index()

In [ ]:
crash_final

In [ ]:
crash_year = crash_df[['year','COLLISION_ID','Killed','Injured']]\
                .groupby('year').agg({'COLLISION_ID':'count','Killed':'sum','Injured':'sum'})\
                .reset_index()\
                .rename(columns={'COLLISION_ID':'Number of Collsion'})
crash_year = crash_year[crash_year['year']!=2021]

In [ ]:
plt.figure(figsize=(10,5))
plt.bar(crash_year.year,crash_year['Number of Collsion'],color='lightblue')
plt.title('Number of Collisions from 2012 to 2020',fontsize=20)
plt.ylabel('Collisions',fontsize=15)
plt.xticks(crash_year.year,fontsize=13)
plt.yticks(fontsize=13)
plt.grid(linestyle='--',axis='y')
plt.savefig('collisions',dpi=300)
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
plt.bar(crash_year.year,crash_year.Injured,color='lightblue')
plt.title('Number of Injuries from 2012 to 2020',fontsize=20)
plt.ylabel('Injuries',fontsize=15)
plt.xticks(crash_year.year,fontsize=13)
plt.yticks(fontsize=13)
plt.grid(linestyle='--',axis='y')
plt.savefig('injuries',dpi=300)
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
plt.bar(crash_year.year,crash_year.Killed,color='lightblue')
plt.title('Number of Fatalities from 2012 to 2020',fontsize=20)
plt.ylabel('Fatalities',fontsize=15)
plt.xticks(crash_year.year,fontsize=13)
plt.yticks(fontsize=13)
plt.grid(linestyle='--',axis='y')
plt.savefig('fatalities',dpi=300)
plt.show()

In [ ]:
missing_value_crash = crash_df.isna().sum().to_frame(name='NaN_number')
missing_value_crash.sort_values('NaN_number',inplace=True,ascending=False)
missing_value_crash.plot.bar(color='blue',figsize=(15,7))
plt.title('Missing Value of Motor Vehicle Collisions dataset',fontsize=15)
plt.legend()
plt.show()

In [3]:
permit_df = pd.read_csv('./data/Street_Construction_Permits.csv',low_memory=False)

In [ ]:
permit_df.isna().sum()

In [ ]:
permit_df.head()

In [4]:
permit = permit_df[(permit_df['BoroughName'].notna()) & (permit_df['OnStreetName'].notna())\
                & (permit_df['IssuedWorkStartDate'].notna()) & (permit_df['IssuedWorkEndDate'].notna())].copy()

In [ ]:
permit.shape

In [ ]:
permit = permit[['PermitNumber','BoroughName','OnStreetName','IssuedWorkStartDate','IssuedWorkEndDate']].reset_index(drop=True)

In [ ]:
permit.head()

In [ ]:
crash_final

In [ ]:
list(set(permit['OnStreetName'].to_list()))

In [5]:
permit.to_csv('permit.csv',index=False)

In [ ]:
# permit_dd = dd.read_csv('./data/Street_Construction_Permits.csv',dtype=str)
# permit_dd['IssuedWorkStartDate'] = dd.to_datetime(permit_dd['IssuedWorkStartDate'],errors = 'coerce')
# permit_dd['IssuedWorkEndDate'] = dd.to_datetime(permit_dd['IssuedWorkEndDate'],errors = 'coerce')
# permit_dd = permit_dd.dropna(subset=['BoroughName','OnStreetName','IssuedWorkStartDate','IssuedWorkEndDate'])
# permit_dd = permit_dd[['BoroughName','OnStreetName','IssuedWorkStartDate','IssuedWorkEndDate']]
# permit_dd['year'] = permit_dd['IssuedWorkStartDate'].dt.year
# permit_dd= permit_dd[(permit_dd['year']>=2016) & (permit_dd['year']<2020)]
# permit_dd['IssuedWorkStartDate'] = permit_dd['IssuedWorkStartDate'].dt.date
# permit_dd['IssuedWorkEndDate'] = permit_dd['IssuedWorkEndDate'].dt.date
# permit_dd.head()

In [ ]:
missing_value_permit = permit_df.isna().sum().to_frame(name='NaN_number')
missing_value_permit.sort_values('NaN_number',inplace=True,ascending=False)
missing_value_permit.plot.bar(color='blue',figsize=(15,7))
plt.title('Missing Value of Street Construction Permit dataset',fontsize=15)
plt.legend()
plt.show()

In [ ]:
permit_dd = dd.read_csv('./data/Street_Construction_Permits.csv',dtype=str)
permit_dd['CreatedOn'] = dd.to_datetime(permit_dd['CreatedOn'].str[:10])

permit_dd['dow'] = dd.to_datetime(permit_dd['CreatedOn']).dt.dayofweek
permit_dd['day'] = dd.to_datetime(permit_dd['CreatedOn']).dt.day
permit_dd['year'] = dd.to_datetime(permit_dd['CreatedOn']).dt.year

permit_year = permit_dd[['year','PermitNumber']].groupby('year').count().reset_index().compute()
permit_year['year']= permit_year[permit_year['year'] != '2021']
permit_year = permit_year[0:-1]
permit_year.year=permit_year['year'].astype(str)
permit_year.year = permit_year.year.str[:4]
permit_year = permit_year[1:].sort_values('year')

# permit = permit_dd.compute()

In [ ]:
plt.figure(figsize=(10,5))
plt.bar(permit_year.year,permit_year['PermitNumber'],color='lightblue')
plt.title('Number of Construction Permits from 2014 to 2020',fontsize=20)
plt.ylabel('Number of Permits',fontsize=15)
plt.xticks(permit_year.year,fontsize=13)
plt.yticks(fontsize=13)
plt.grid(linestyle='--',axis='y')
plt.savefig('permits',dpi=300)
plt.show()

In [ ]:
# number of permits per year
permit_year['PermitNumber'].sum()/len(permit_year['PermitNumber'])

In [6]:
import pyspark
sc = pyspark.SparkContext()

In [7]:
def data_time_range(record):
    import datetime
    split_sdate = record[4].split(' ')[0].split('/')
    start = (split_sdate[2],split_sdate[0],split_sdate[1])
    start = datetime.datetime(*map(int, start ))
    
    split_edate = record[5].split(' ')[0].split('/')
    end = (split_edate[2],split_edate[0],split_edate[1])
    end = datetime.datetime(*map(int, end ))
    return (record[2],record[3],start,end)
def year_filter(record):
    import datetime
    sd = datetime.datetime(2016,1,1)
    ed = datetime.datetime(2019,12,31)
    if record[2] > sd and record[3] < ed:
        return True
    else:
        return False
def range_of_date(record):
    from datetime import date, datetime, timedelta
    l = []
    for result in perdelta(record[2], record[3], timedelta(days=1)):
        l.append((result,1))
    res_tuple = []
    for i in l:
        res_tuple.append(i)
    return ((record[0],record[1]),res_tuple)

def perdelta(start, end, delta):
    curr = start
    while curr <= end:
        yield curr
        curr += delta

# def date_count(record):
#     import pyspark
#     spark = pyspark.SparkContext()
#     date_list = spark.parallelize(record[1]).reduceByKey(lambda x,y: x+y).collect()
#     return (record[0],date_list)

def date_count(record):
    d_list = record[1]
    first = itemgetter(0)
    sums = [(k, sum(item[1] for item in tups_to_sum))
        for k, tups_to_sum in groupby(sorted(d_list, key=first), key=first)]
    return [record[0],sums]

def output(record):
    return [record[0][0],record[0][1],record[1]]
    
final_list = sc.textFile('permit.csv')\
    .map(lambda x: next(csv.reader([x])))\
    .filter(lambda x: x[1] != 'PermitNumber')\
    .map(data_time_range)\
    .filter(year_filter)\
    .map(range_of_date)\
    .reduceByKey(lambda x,y: x+y)\
    .map(date_count)\
    .map(output).collect()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 6 in stage 0.0 failed 1 times, most recent failure: Lost task 6.0 in stage 0.0 (TID 6) (192.168.1.144 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/valx/anaconda3/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 604, in main
    process()
  File "/home/valx/anaconda3/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 594, in process
    out_iter = func(split_index, iterator)
  File "/home/valx/anaconda3/lib/python3.8/site-packages/pyspark/rdd.py", line 2916, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/home/valx/anaconda3/lib/python3.8/site-packages/pyspark/rdd.py", line 2916, in pipeline_func
    return func(split, prev_func(split, iterator))
  Fi

In [ ]:
# range of dates from 2016-1-1 to 2019-12-31
date_col = []
for result in perdelta(datetime(2016,1,1),datetime(2019,12,31), timedelta(days=1)):
        date_col.append(result)
len(date_col)

In [ ]:
# create and finalize the dataframe
final_list = np.array(final_list,dtype=object)
permit_df_final = pd.DataFrame(final_list[:,:2],columns=['borough','street'])
for col in date_col:
    permit_df_final[col] = 0
dates = final_list[:,2]
for i, row in permit_df_final.iterrows():
    range_date = dates[i]
    for a in range_date:
        permit_df_final.at[i,a[0]]=a[1]
permit_df_final['key']=list(zip(permit_df_final['borough'],permit_df_final['street']))
first_column = permit_df_final.pop('key')
permit_df_final.insert(0, 'key', first_column)

In [ ]:
# count total number of constructions for each street
permit_final = permit_df_final.copy()
permit_final['total_constructions']=permit_final.iloc[:,3:].sum(axis=1)
column = permit_final.pop('total_constructions')
permit_final.insert(3, 'total_constructions', column)
permit_final

In [ ]:
# permit_df_final.to_csv('permit_final.csv',index=False)

In [ ]:
# permit_boro_st = permit_df_final.drop(columns=['borough','street']).copy()

In [ ]:
permit_df_final = permit_final.copy()

In [ ]:
permit_df_final.drop(columns=['borough','street'],inplace=True)

In [ ]:
permit_df_final.head()

In [ ]:
crash_permit = crash_final.merge(permit_df_final,left_on='key',right_on='key')

In [ ]:
column = crash_permit.pop('key')
crash_permit.insert(0, 'key', column)
crash_permit

In [ ]:
crash_permit.to_csv('crash_permit.csv',index=False)
df_test = pd.read_csv('crash_permit.csv')
df_test.iloc[:,1:].astype(float)

In [ ]:
df_test['key'].dtype

In [ ]:
cp_no_zero = df_test[df_test['NUMBER OF PERSONS INJURED'] != 0 ].copy()
cp_no_zero['safety_index'] = cp_no_zero['total_constructions']/\
                                (cp_no_zero['NUMBER OF PERSONS KILLED'] + cp_no_zero['NUMBER OF PERSONS INJURED'])

In [ ]:
X = cp_no_zero.iloc[:,-1:]

In [ ]:
cluster_range = range(2,5)
for n_c in cluster_range:
    silhouette = []
    for i in range(10):
        km = KMeans(random_state=i*i, n_clusters=n_c)
        cluster_labels = km.fit(X).labels_
        silhouette.append(silhouette_score(X, cluster_labels))
    silhouette_avg = np.mean(silhouette)
    print("For n_clusters = {},".format(n_c)+" the average silhouette_score is :{}".format(silhouette_avg))

In [ ]:
# crash_permit['Injured'].replace({0:1e-1},inplace=True)
# crash_permit['safety_index'] = crash_permit['total_constructions']/\
#                                 (crash_permit['Killed'] + crash_permit['Injured'])

In [ ]:
max(crash_permit['safety_index'])

In [ ]:
min(crash_permit['safety_index'])

In [ ]:
# crash_permit.isna().sum()

In [ ]:
crash_permit.columns[:12]

In [ ]:
crash_permit.iloc[:,1:12]

In [ ]:
crash_permit[crash_permit['key']== ('MANHATTAN', 'BROADWAY')] ['total_constructions']

In [ ]:
crash_permit[crash_permit['total_constructions']==0]

In [ ]:
crash_permit.iloc[crash_permit['total_constructions'].idxmin()]

In [ ]:
X = cp_no_zero.iloc[:,1:12]
# X.drop(columns=['Killed'],inplace = True)

In [ ]:
cp_no_zero.head()

In [ ]:
# constructions/(Killed * 10 + Injuries)  Injuries = 0, replace = 1e-5

In [ ]:
# K-means, TODO: gridSearchCV()
cluster_range = range(2,5)
for n_c in cluster_range:
    silhouette = []
    for i in range(100):
        km = KMeans(random_state=i*i, n_clusters=n_c)
        cluster_labels = km.fit(X).labels_
        silhouette.append(silhouette_score(X, cluster_labels))
    silhouette_avg = np.mean(silhouette)
    print("For n_clusters = {},".format(n_c)+" the average silhouette_score is :{}".format(silhouette_avg))

In [ ]:
# the best K-means model
km = KMeans(random_state=2020, n_clusters=2)
kmeans_labels = km.fit(X).labels_

In [ ]:
cp_kmeans = crash_permit.copy()
cp_kmeans['kmeans_label'] = kmeans_labels

In [ ]:
cp_kmeans.columns[:13]

In [ ]:
min(cp_kmeans['Injured'].to_list())

In [ ]:
len(cp_kmeans[cp_kmeans['Injured']==0]['total_constructions'])

In [ ]:
len(cp_kmeans[cp_kmeans['Killed']==0]['total_constructions'])

In [ ]:
cp_kmeans[cp_kmeans['kmeans_label'] == 0]

In [ ]:
# GMM
for n_clusters in cluster_range:
    silhouette = []
    for i in range(100):
        gm = GaussianMixture(random_state=i*i, n_components=n_clusters)
        cluster_labels = gm.fit(X).predict(X)
        silhouette.append(silhouette_score(X, cluster_labels))
    silhouette_avg = np.mean(silhouette)
    print("For n_clusters = {},".format(n_c)+" the average silhouette_score is :{}".format(silhouette_avg))